## Tools

In [ ]:
%pip install tensorflow[and-cuda]

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
sns.set_style("darkgrid")

## Dataset

In [ ]:
# Dataset readin

## Histogram Creation

In [ ]:
#todo

## Naive Bayes Implementation

In [ ]:
#todo